In [ ]:
!pip install neo4j

In [1]:
from neo4j import GraphDatabase

In [2]:
import pandas as pd

In [3]:
class Neo4jConnection:    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [4]:
def extract_data(quer_result):
    temp_dict = {}
    keys = quer_result[0].keys()
    for key in keys:
        temp_dict[key] = []
    attr_count = len(keys)
    for record in quer_result:
        #print(record)
        for i in range(attr_count):
            temp_dict[keys[i]].append(record[keys[i]])
    #print(temp_dict)
    return pd.DataFrame(temp_dict)          

##### Creating Connection Object

In [5]:
conn = Neo4jConnection(uri="bolt://localhost:11003", 
                       user="neo4j",              
                       pwd="1234")

##### Delete existing nodes

In [6]:
query0 = '''MATCH(n) DETACH DELETE n'''
conn.query(query0)

[]

##### Creating Products graph nodes

In [7]:
query1 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/products.csv" AS row
CREATE (n:Product)
SET n = row,
n.unitPrice = toFloat(row.unitPrice),
n.unitsInStock = toInteger(row.unitsInStock), 
n.unitsOnOrder = toInteger(row.unitsOnOrder),
n.reorderLevel = toInteger(row.reorderLevel), 
n.discontinued = (row.discontinued <> "0")
'''

In [8]:
conn.query(query1)

[]

##### Creating Categories graph nodes

In [9]:
q2 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/categories.csv" AS row
CREATE (n:Category)
SET n = row'''

In [10]:
conn.query(q2)

[]

##### Creating Suppliers graph nodes

In [11]:
q3 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/suppliers.csv" AS row
CREATE (n:Supplier)
SET n = row'''

In [12]:
conn.query(q3)

[]

##### Creating Indices

In [13]:
q4 = '''CREATE INDEX FOR (p:Product) ON (p.productID)'''
q5 = '''CREATE INDEX FOR (p:Product) ON (p.productName)'''
q6 = '''CREATE INDEX FOR (c:Category) ON (c.categoryID)'''

In [14]:
conn.query(q4)
conn.query(q5)
conn.query(q6)

Query failed: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=3, name='index_eadfd5a0', type='GENERAL BTREE', schema=(:Product {productID}), indexProvider='native-btree-1.0' )'.}
Query failed: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=4, name='index_60a51174', type='GENERAL BTREE', schema=(:Product {productName}), indexProvider='native-btree-1.0' )'.}
Query failed: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=5, name='index_44f4e370', type='GENERAL BTREE', schema=(:Category {categoryID}), indexProvider='native-btree-1.0' )'.}


##### Creating Relations between category and product nodes

In [15]:
q7 = '''MATCH (p:Product),(c:Category)
WHERE p.categoryID = c.categoryID
CREATE (p)-[:PART_OF]->(c)'''

In [16]:
conn.query(q7)

[]

##### Creating Relation between supplier and product nodes

In [17]:
q8 = '''MATCH (p:Product),(s:Supplier)
WHERE p.supplierID = s.supplierID
CREATE (s)-[:SUPPLIES]->(p)'''

In [18]:
conn.query(q8)

[]

##### Query to retrieve all suppliers, and respective supplied product categories

In [19]:
q9 = '''MATCH (s:Supplier)-->(:Product)-->(c:Category)
RETURN s.companyName as Company, collect(distinct c.categoryName) as Categories'''

In [20]:
quer9 = conn.query(q9)

In [21]:
quer9[0]

<Record Company='Bigfoot Breweries' Categories=['Beverages']>

In [22]:
#type(quer)

In [ ]:
#type(quer[0])

In [ ]:
#quer[0]['Categories']

In [ ]:
#quer[0]['Company']

In [ ]:
#quer[0].keys()

In [23]:
extract_data(quer9)

,Company,Categories
0,Bigfoot Breweries,[Beverages]
1,Pavlova,"[Beverages, Condiments, Confections, Meat/Poul..."
2,Aux joyeux ecclésiastiques,[Beverages]
3,Karkki Oy,"[Beverages, Confections]"
4,Leka Trading,"[Beverages, Condiments, Grains/Cereals]"
5,Plutzer Lebensmittelgroßmärkte AG,"[Beverages, Condiments, Grains/Cereals, Meat/P..."
6,Refrescos Americanas LTDA,[Beverages]
7,Exotic Liquids,"[Beverages, Condiments]"
8,Forêts d'érables,"[Condiments, Confections]"
9,New Orleans Cajun Delights,[Condiments]


#####  Query to retrieve distinct suppliers

In [25]:
q10 = '''MATCH (c:Category {categoryName:"Produce"})<--(:Product)<--(s:Supplier)
RETURN DISTINCT s.companyName as ProduceSuppliers'''

In [26]:
quer10 = conn.query(q10)
extract_data(quer10)

,ProduceSuppliers
0,G'day
1,Tokyo Traders
2,Plutzer Lebensmittelgroßmärkte AG
3,Mayumi's
4,Grandma Kelly's Homestead


##### Query to load customers database

In [27]:
q11 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/customers.csv" AS row
CREATE (n:Customer)
SET n = row'''

In [28]:
conn.query(q11)

[]

##### Query to load orders database

In [29]:
q12 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/orders.csv" AS row
CREATE (n:Order)
SET n = row'''

In [30]:
conn.query(q12)

[]

##### Query to create indices

In [31]:
q13 = '''CREATE INDEX FOR (n:Customer) ON (n.customerID)'''
q14 = '''CREATE INDEX FOR (o:Order) ON (o.orderID)'''

In [32]:
conn.query(q13)
conn.query(q14)

Query failed: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=6, name='index_22b7066a', type='GENERAL BTREE', schema=(:Customer {customerID}), indexProvider='native-btree-1.0' )'.}
Query failed: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=7, name='index_9cd40186', type='GENERAL BTREE', schema=(:Order {orderID}), indexProvider='native-btree-1.0' )'.}


##### Query to create relations between customer and order databases

In [33]:
q15 = '''MATCH (n:Customer),(o:Order)
WHERE n.customerID = o.customerID
CREATE (n)-[:PURCHASED]->(o)
'''

In [34]:
conn.query(q15)

[]

##### Query to load order-details database

In [35]:
q16 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/order-details.csv" AS row
MATCH (p:Product), (o:Order)
WHERE p.productID = row.productID AND o.orderID = row.orderID
CREATE (o)-[details:ORDERS]->(p)
SET details = row,
details.quantity = toInteger(row.quantity)'''

In [36]:
conn.query(q16)

[]

##### Query to retriee total products purchased by individual customers

In [38]:
q17 = '''MATCH (cust:Customer)-[:PURCHASED]->(:Order)-[o:ORDERS]->(p:Product),
  (p)-[:PART_OF]->(c:Category {categoryName:"Produce"})
RETURN DISTINCT cust.contactName as CustomerName, SUM(o.quantity) AS TotalProductsPurchased'''

In [39]:
quer17 = conn.query(q17)
extract_data(quer17)

,CustomerName,TotalProductsPurchased
0,Maria Larsson,148
1,Hanna Moos,11
2,Mario Pontes,35
3,Isabel de Castro,18
4,Carine Schmitt,3
...,...,...
58,Yoshi Latimer,15
59,Howard Snyder,3
60,Yvonne Moncada,18
61,Ann Devon,100


##### Query to return most active customer

In [40]:
q18 = '''MATCH (cust:Customer)-[:PURCHASED]->(:Order)-[o:ORDERS]->(p:Product),
  (p)-[:PART_OF]->(c:Category {categoryName:"Produce"})
RETURN DISTINCT cust.contactName as CustomerName, SUM(o.quantity) AS TotalProductsPurchased
ORDER BY TotalProductsPurchased DESC'''

In [41]:
quer18 = conn.query(q18)
extract_data(quer18)

,CustomerName,TotalProductsPurchased
0,Roland Mendel,353
1,Horst Kloss,242
2,Jose Pavarotti,152
3,Maria Larsson,148
4,Patricia McKenna,121
...,...,...
58,Rita Müller,4
59,Carine Schmitt,3
60,Matti Karttunen,3
61,Ana Trujillo,3


##### Most Popular Customer

In [42]:
extract_data(quer18).head(1)

,CustomerName,TotalProductsPurchased
0,Roland Mendel,353
